In [181]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [182]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_drop_df = application_df.drop(columns=['EIN','NAME'])



In [183]:
# Determine the number of unique values in each column.
application_drop_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [184]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_drop_df['APPLICATION_TYPE'] = application_drop_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_drop_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [185]:
# Look at CLASSIFICATION value counts for binning
classification = application_drop_df['CLASSIFICATION'].value_counts()
classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [186]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_greater = [i for i in classification if i > 1]
classification_greater

[17326,
 6074,
 4837,
 1918,
 1883,
 777,
 287,
 194,
 116,
 114,
 104,
 95,
 75,
 58,
 50,
 36,
 34,
 32,
 32,
 30,
 20,
 18,
 16,
 15,
 15,
 14,
 11,
 10,
 10,
 9,
 9,
 7,
 6,
 6,
 6,
 5,
 5,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2]

In [187]:
# Create list of classifications to place in bin
classification_filter = []
for i in range(len(classification)):
  if classification[i] < 1883:
    classification_filter.append(classification.index[i])

In [188]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_filter

# Replace in dataframe
for cls in classifications_to_replace:
    application_drop_df['CLASSIFICATION'] = application_drop_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_drop_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [189]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_affiliation = pd.get_dummies(application_drop_df['AFFILIATION'])
dummy_use_case = pd.get_dummies(application_drop_df['USE_CASE'])
dummy_organization = pd.get_dummies(application_drop_df['ORGANIZATION'])
dummy_special_considerations = pd.get_dummies(application_drop_df['SPECIAL_CONSIDERATIONS'])
dummy_classification = pd.get_dummies(application_drop_df['CLASSIFICATION'])
dummy_application_type = pd.get_dummies(application_drop_df['APPLICATION_TYPE'])
dummy_income_amt = pd.get_dummies(application_drop_df['INCOME_AMT'])
application_dummy_df = application_drop_df.drop(columns=['AFFILIATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS','CLASSIFICATION','APPLICATION_TYPE','INCOME_AMT'])
application_dummy_df = pd.concat([application_dummy_df,dummy_affiliation,dummy_use_case,dummy_organization,dummy_special_considerations,dummy_application_type,dummy_classification,dummy_income_amt], axis=1)
application_dummy_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,CompanySponsored,Family/Parent,Independent,National,Other,Regional,CommunityServ,...,Other,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [190]:
# Split our preprocessed data into our features and target arrays
y = application_dummy_df['IS_SUCCESSFUL'].values
X = application_dummy_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [191]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [192]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 5)                 220       
                                                                 
 dense_27 (Dense)            (None, 4)                 24        
                                                                 
 dense_28 (Dense)            (None, 1)                 5         
                                                                 
Total params: 249 (996.00 Byte)
Trainable params: 249 (996.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [193]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [194]:
# Train the model
fit = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6134 - accuracy: 0.6914
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5749 - accuracy: 0.7240
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5679 - accuracy: 0.7261
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5642 - accuracy: 0.7277
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5603 - accuracy: 0.7287
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7289
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5567 - accuracy: 0.7295
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5558 - accuracy: 0.7294
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7296
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accura

In [195]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5559 - accuracy: 0.7277 - 464ms/epoch - 2ms/step
Loss: 0.5559422969818115, Accuracy: 0.7276967763900757


In [196]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
